In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [2]:
from pyspark.sql import SparkSession, SQLContext
import os
import pyspark.sql.functions as f
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

22/04/12 18:34:29 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/12 18:34:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/12 18:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = SparkSession.builder.appName('App Name').master("local[*]").getOrCreate()
sqlContext = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)

In [4]:
data = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath("data/classics_clean.csv"))

In [5]:
import nltk
nltk.pos_tag(["Arthur", "loves", "fish"])

[('Arthur', 'NNP'), ('loves', 'VBZ'), ('fish', 'JJ')]

In [6]:
import pyspark.sql.functions as f
data = data.withColumn('line', f.explode(f.split(f.col('text'), '\n')))
data.show()

+----+----+--------------------+--------------------+
|  id|type|                text|                line|
+----+----+--------------------+--------------------+
|1905|   C| the governess or...| the governess or...|
| 768|   C| transcribed from...| transcribed from...|
|1342|   C| pride and prejud...| pride and prejud...|
|1260|   C| transcribed from...| transcribed from...|
| 514|   C| little women by ...| little women by ...|
| 113|   C| in honor of lisa...| in honor of lisa...|
| 145|   C| middlemarch by g...| middlemarch by g...|
|  45|   C| anne of green ga...| anne of green ga...|
+----+----+--------------------+--------------------+



In [7]:
def removeProperNouns(sentence):
    tagged_sentence = nltk.tag.pos_tag(nltk.word_tokenize(sentence))
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    return ' '.join(edited_sentence)

In [8]:
proper_nouns_removed = data.select(f.col("text")).rdd.map(removeProperNouns)